# Anomaly Detection - Zelong
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>

<br>

**Dennis Milechin, Ivan Sunyagin, Hany Bassily**

<br>
## Beta Version for Hurricane Matthew


<br>

Import libraries

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import time
from scipy.integrate import simps

#import pydotplus
from IPython import display
from IPython.display import Image
#import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("paper")
#sns.set_palette("RdBu", n_colors=32)
#from inspect import getmembers

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors

from scipy import stats
from scipy.stats import gaussian_kde as PDF

import collections
from matplotlib import rcParams
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

**---**

## Open a datasets

<br>

In [ ]:
# Open data sets:
# --------------

# Data dictionary
data_dic = {}

# specify date columns
date_range = [[0,1,2,3,4]]

# Open the data files
for i in range(5):
    file_name = 'datasets/data_' + str(i+1) + '.txt'
    inter = pd.read_csv(file_name , delim_whitespace = True , skiprows = [1],
                                              parse_dates = date_range , infer_datetime_format = True)
    data_dic['df_' + str(i + 1)] = inter.iloc[:,1:]

# Sanity check    
data_dic['df_1'].head(10)

In [ ]:
# Concatenate all files:
# ---------------------

# initiation
df_global_raw = data_dic['df_1']

# concatente

title = ['WSPD' , 'GST' , 'PRES' , 'ATMP']
# title = ['WDIR' , 'WSPD' , 'GST' , 'PRES' , 'ATMP' , 'DEWP']

for i in range(2,6):
    df_global_raw = pd.concat([df_global_raw , data_dic['df_' + str(i)]] , axis = 0)
    
df_global = df_global_raw[title]

# data array
data_raw = df_global.values

In [ ]:
# Preprocessing
# -------------

# determine defected rows
defect = []

# itterrate
for i in range (data_raw.shape[0]):
    if data_raw[i,-1] > 200.:
        defect.append(i)

# remove rows
data_clean = np.delete(data_raw, defect, axis = 0)

## General Functions

<br>

### 1- Power Martingale with defined Epsilon

<br>

In [ ]:
# Function to calculate Power Martingale - Single:
# ----------------------------------------------

def power_martingale(data , eps , delay, sim = False):
    
    # Normalization
    data_norm = data / np.std(data, axis = 0)
    
    # initiate Filtered data
    d = np.zeros(data_norm.shape)
    
    # delay-Filter
    for i in range (delay, data_norm.shape[0]):
        d[i, :] = data_norm[i,:] - data_norm[ i - delay , : ]
        
    if(sim):
        d = data
 
    # data mean
    mean = np.average(d , axis = 0)
    
    # mean vector norm
    mean_norm = np.linalg.norm(mean)

    # strangenes reference
    ref_alpha = np.linalg.norm(d.std(axis = 0)) + mean_norm

    # Length of data
    L = d.shape[0]

    # alpha
    alpha = np.linalg.norm(d , axis = 1) / ref_alpha

    # p-value(randomised)
    p_value = np.ones(alpha.shape)

    # iterrate for p-value calculation
    for i in range(1,L):
    
        # end value
        end = alpha[i]
    
        # alpha subset
        alpha_sub = alpha[:i + 1]

        # number of elements
        n = i + 1   
    
        # p-values calculation
        np.random.seed(200)
        p_value[i] = ( float(np.sum(alpha_sub > end)) + np.random.uniform() * np.sum(alpha_sub == end ) ) / float(n)
        # p_value[i] = float(np.sum(alpha_sub >= end))  / float(n)
    
        # to avoid log zero
        p_value[p_value==0] = 0.0000000001
    
    # initiate Power Martingale
    power_mart = np.zeros(alpha.shape)

    # dummy multiplier
    M = 0.
    
    # initiate difference measure
    diff_arr = np.zeros((L,))
    
    # Initiate difference multiplier
    diff = 1.

    # iterrate for power martingale calculation
    for i in range(L):
        
        # increment
        delta = eps * (p_value[i]) ** (eps - 1.)
        
        # Calculate martingale difference
        # diff_arr[i] = np.log( np.absolute(diff * (delta - 1.)) )
        diff_arr[i] = np.absolute(np.log(delta))
        
        # Update power martingal
        M += np.log(delta)  
        
        # Array
        power_mart[i] = M
        
    return power_mart , diff_arr

### 2- Power Martingale with Mixture

<br>

In [ ]:
# Function to calculate Power Martingale:
# ---------------------------------------

def power_martingale_mix(data , res , delay, sim = False):
    
    # Normalization
    data_norm = data / np.std(data, axis = 0)
    
    # initiate Filtered data
    d = np.zeros(data_norm.shape)
    
    # delay-Filter
    for i in range (delay, data_norm.shape[0]):
        d[i, :] = data_norm[i,:] - data_norm[ i - delay , : ]
        
    if(sim):
        d = data

    # epsilon array
    eps = np.linspace(0.001, 0.999, res)
    
    # data mean
    mean = np.average(d , axis = 0)
    
    # mean vector norm
    mean_norm = np.linalg.norm(mean)

    # strangenes reference
    ref_alpha = np.linalg.norm(d.std(axis = 0)) + mean_norm

    # Length of data
    L = d.shape[0]

    # alpha
    alpha = np.linalg.norm(d , axis = 1) / ref_alpha

    # p-value(randomised)
    p_value = np.ones(alpha.shape)

    # iterrate for p-value calculation
    for i in range(1,L):
    
        # end value
        end = alpha[i]
    
        # alpha subset
        alpha_sub = alpha[:i - 1]

        # number of elements
        n = i + 1   
    
        # p-values calculation
        np.random.seed(200)
        p_value[i] = ( float(np.sum(alpha_sub > end)) + np.random.uniform() * np.sum(alpha_sub == end ) ) / float(n)
        # p_value[i] = float(np.sum(alpha_sub >= end))  / float(n)
    
        # to avoid log zero
        p_value[p_value==0] = 0.00000000001
    
    # initiate Power Martingale
    power_mart = np.zeros(alpha.shape)
    
    # initiate difference
    diff_arr = np.zeros(alpha.shape)

    # dummy multiplier
    M = 0.

    # iterrate for power martingale calculation
    for i in range(L):
    
        # Calculate delta Array    
        delta = np.log(eps) + (eps - 1.) * np.log(p_value[i])
        
        # Integrate for mix
        delta_mix = simps(delta , eps)
        
        # update power martingale
        M += delta_mix
        
        # Array
        power_mart[i] = M
        
        # difference
        diff_arr[i] = np.absolute(delta_mix)
        
        
    return power_mart , diff_arr

### 3- Support Functions

<br>

In [ ]:
# Exponential Smoother:
# ---------------------

def smooth(y , alpha):
    
    # initiation of the filtered signal array
    x = np.zeros(y.shape)
    
    # Data size
    L = y.shape[0]
    
    # intiation
    x[0] = 0.5 * (1. + alpha) * y[0]
    
    # Iterrate for new samples
    for i in range(1,L):
        x[i] = alpha * y[i] + (1. - alpha) * x[i - 1]
        
    return x

In [ ]:
# Function to generate a heatmap array for raw and smoothed signal
# ----------------------------------------------------------------

def heatmap_fil(x , alpha):
    
    # signal length
    L = x.shape[0]
    
    # apply smoothing
    xk = smooth(x , alpha)
    
    # reshape arrays
    x1 = xk.reshape((1,L)) / np.max(xk)
    x2 = x.reshape((1,L)) / np.max(x)
    
    # Augment arrays
    big1 = np.concatenate((x1 , x1) , axis = 0)
    big2 = np.concatenate((x2 , x2) , axis = 0)
    
    # Join arrays
    big = np.concatenate((big1 , big2) , axis = 0)
    
    return big
        


## Model Tuning

<br>

### 1- General Constants

<br>

In [ ]:
# Selected epsilon array
e_array = np.linspace(0.01,0.99,100)

# selected delay array
delay_array = np.array(range(1,501))

# Dimensions
rows = len(delay_array)
cols = len(e_array)

# Resolution for mixture integral
resolution = 200

# Smoothing factor for change detection
a = 0.008

# Data size
L = data_clean.shape[0]

# Epsilon Value
eps = 0.307

# Delay Value
delay = 189

# Color map 
clr = plt.get_cmap('jet')

### 2- Power Martingale with variable Epsilon

<br>

In [ ]:
# Application and Visualization
# -----------------------------

# Filtered data
p_mar2_pwr, diff2_pwr = power_martingale(data_clean,eps , delay)

##

In [ ]:
# Visualization of Power Martingale
# ---------------------------------

# plot initialization
fig = plt.figure(figsize = (15,8))

# plot simulated data
ax1 = fig.add_subplot(1,2,1)
ax1.plot(p_mar1_pwr)
ax1.set_title('Simulated Data')

# plot raw data
ax2 = fig.add_subplot(1,2,2)
ax2.plot(p_mar2_pwr)
ax2.set_title(' Processed Data ')

plt.show()

In [ ]:
# Visualize change detection
# --------------------------

fig = plt.figure(figsize = (20,10))

# Simulated Data
ax1 = fig.add_subplot(121)
ax1.plot(smooth(diff1_pwr , a))
ax1.plot(diff1_pwr , alpha = 0.5)
ax1.set_title('Simulated Data')

# Processed data data
ax2 = fig.add_subplot(122)
ax2.plot(smooth(diff2_pwr , a) )
ax2.plot(diff2_pwr , alpha = 0.5)
ax2.set_title('Filtered Data')

plt.show()

In [ ]:
# visualize the heatmaps for the change detection (CD):
# ----------------------------------------------------

fig = plt.figure(figsize = (20,10))

#
ht1_pwr = heatmap_fil(diff1_pwr , a)
ax1 = fig.add_subplot(2,1,1)
ax1.text(6550,1,'Smoothed CD')
ax1.text(6550,3,'Raw CD')
ax1.set_title('Simulated')
ax1.pcolor(ht1_pwr , cmap = clr)
ax1.axes.get_yaxis().set_visible(False)

#
ht2_pwr = heatmap_fil(diff2_pwr , a)
ax2 = fig.add_subplot(2,1,2)
ax2.text(22500,1,'Smoothed CD')
ax2.text(22500,3,'Raw CD')
ax2.set_title('Actual')
ax2.pcolor(ht2_pwr , cmap = clr)
ax2.axes.get_yaxis().set_visible(False)

plt.show()

### 2- Power Martingale with Mixture

<br>

In [ ]:
# Tune for delay:
#----------------

# Initiation of measure array
measure_mix = np.zeros((rows,))

# timer initiation
timer_2 = 0.

for i in range(rows):
    
    # start timer
    start_2 = time.time()
    
    # specify delsy
    de = int(delay_array[i])
    
    # display progress
    display.clear_output(wait=True)
    display.display('Processing delay of ' + str(de) + ' at step ' + str(i + 1), 
                    'Process time : ' + str(timer_2) + ' sec')
        
    # evaluate the difference
    pmar, dif = power_martingale_mix(data_clean, resolution , de)
    dif_s = smooth(dif , a)
    
        
    # maximum data Matthew
    dif_max_m = np.max(dif_s[17000:18000])
    
        
    # Put in measure array
    measure_mix[i] = (dif_max_m - np.average(dif_s)) / dif_s.std(axis=0)
    
    # stop timer
    timer_2 = time.time() - start_2


                   

In [ ]:
# Viisualize Tunning results:
# ---------------------------

fig = plt.figure(figsize = (10,10))

ax = fig.add_subplot(111)
ax.plot(measure_mix)

plt.show()

In [ ]:
# Best Values
# -----------

best_de_mix = int(delay_array[np.argmax(measure_mix)])

print '\nBest Delay is : ' , best_de_mix

In [ ]:
# Application and Visualization (Power Martingale with Mixture)
# -------------------------------------------------------------

# simulated data 
p_mar1_mix , diff1_mix = power_martingale_mix(sim_data,resolution , best_de_mix , sim = True)

# Filtered data
p_mar2_mix, diff2_mix = power_martingale_mix(data_clean,resolution , best_de_mix)

##

In [ ]:
# Visualization of Power Martingale
# ---------------------------------

# plot initialization
fig = plt.figure(figsize = (15,8))

# plot simulated data
ax1 = fig.add_subplot(1,2,1)
ax1.plot(p_mar1_mix)
ax1.set_title('Simulated Data')

# plot raw data
ax2 = fig.add_subplot(1,2,2)
ax2.plot(p_mar2_mix)
ax2.set_title(' Processed Data ')

plt.show()

In [ ]:
# Visualize change detection
# --------------------------

fig = plt.figure(figsize = (20,10))

# Simulated Data
ax1 = fig.add_subplot(121)
ax1.plot(smooth(diff1_mix , a))
ax1.plot(diff1_mix , alpha = 0.5)
ax1.set_title('Simulated Data')

# Processed data data
ax2 = fig.add_subplot(122)
ax2.plot(smooth(diff2_mix , a) )
ax2.plot(diff2_mix , alpha = 0.5)
ax2.set_title('Filtered Data')

plt.show()

In [ ]:
# visualize the heatmaps for the change detection (CD):
# -----------------------------------------------------

fig = plt.figure(figsize = (20,10))

#
ht1_mix = heatmap_fil(diff1_mix , a)
ax1 = fig.add_subplot(2,1,1)
ax1.pcolor(ht1_mix , cmap = clr)
ax1.text(6550,1,'Smoothed CD')
ax1.text(6550,3,'Raw CD')
ax1.set_title('Simulated')
ax1.axes.get_yaxis().set_visible(False)

#
ht2_mix = heatmap_fil(diff2_mix , a)
ax2 = fig.add_subplot(2,1,2)
ax2.pcolor(ht2_mix , cmap = clr)
ax2.text(22500,1,'Smoothed CD')
ax2.text(22500,3,'Raw CD')
ax2.set_title('Actual')
ax2.axes.get_yaxis().set_visible(False)

plt.show()

## Summary of two methods

<br>

### 1- Processed Data Plot

<br>
**Data after normalization and applying the best delay difference**

<br>

In [ ]:
# Preprocess the data with normalization and applying best delay difference:
# --------------------------------------------------------------------------

# Select best delay to apply
best_de = best_de_pwr

# Normalization
data_preproc = data_clean / np.std(data_clean, axis = 0)
    
# initiate Filtered data
data_preproc_filtered = np.zeros(data_preproc.shape)
    
# delay-Filter
for i in range (best_de, data_preproc.shape[0]):
    data_preproc_filtered[i, :] = data_preproc[i,:] - data_preproc[ i - best_de , : ]

In [ ]:
# Visualize the preprocessed data:
# --------------------------------

fig = plt.figure(figsize = (20, 30))
n = data_preproc_filtered.shape[1]

# iterrate for data columns
for i in range (n):
    ax = fig.add_subplot(n,1,i + 1)
    ax.plot(data_preproc_filtered[:,i])
    ax.set_title(title[i])
    
plt.show()

### 2- Power and Mix Martingale Plots

<br>

In [ ]:
# Visualization of Power Martingale
# ---------------------------------

# plot initialization
fig = plt.figure(figsize = (15,8))

# plot simulated data
ax1 = fig.add_subplot(1,2,1)
ax1.plot(p_mar1_pwr , label = 'Power Martingale')
ax1.plot(p_mar1_mix , label = 'Mix Martingale')
ax1.set_title('Simulated Data')
ax1.legend()

# plot raw data
ax2 = fig.add_subplot(1,2,2)
ax2.plot(p_mar2_pwr , label = 'Power Martingale')
ax2.plot(p_mar2_mix , label = 'Mix Martingale')
ax2.set_title(' Processed Data ')
ax2.legend()

plt.show()

### 3- Change Detection Plots

<br>

In [ ]:
# Visualize change detection
# --------------------------

fig = plt.figure(figsize = (20,10))

# Simulated Data
ax1 = fig.add_subplot(121)
ax1.plot(smooth(diff1_pwr , a) , label = 'Power Martingale', alpha = 0.5)
ax1.plot(smooth(diff1_mix , a) , label = 'Mix Martingale' , alpha = 0.9)
ax1.set_title('Simulated Data')
ax1.legend()

# Processed data data
ax2 = fig.add_subplot(122)
ax2.plot(smooth(diff2_pwr , a) , label = 'Power Martingale', alpha = 0.5)
ax2.plot(smooth(diff2_mix , a) , label = 'Mix Martingale' , alpha = 0.9)
ax2.set_title('Filtered Data')
ax2.legend()

plt.show()

### 4- Heatmap Plots

<br>

In [ ]:
# Comparitive heatmap array
# -------------------------
    
# apply smoothing
xk1 = smooth(diff2_pwr , a)
xk2 = smooth(diff2_mix , a)
    
# reshape arrays
x1 = xk1.reshape((1,L)) #/ np.max(xk1)
x2 = xk2.reshape((1,L)) #/ np.max(xk2)
    
# Augment arrays
ext1 = np.concatenate((x1 , x1) , axis = 0)
ext2 = np.concatenate((x2 , x2) , axis = 0)
    
# Join arrays
ext = np.concatenate((ext1 , ext2) , axis = 0)

        

In [ ]:
# visualize the heatmaps:
# -----------------------

fig = plt.figure(figsize = (20,4))

#
ax = fig.add_subplot(1,1,1)
ax.pcolor(ext , cmap = clr)
ax.text(22500,1,'Power Martingale')
ax.text(22500,3,'Mix Martingale')
ax.axes.get_yaxis().set_visible(False)

plt.show()

---